In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 631.6 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=b3539e46a074e66f7d0d7568aebabd3ab4895b280ca00c7b26e90cd00b713e36
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
column_names = raw_datasets["train"].column_names
print(column_names)

features = raw_datasets["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [4]:
text_column_name = "tokens"
label_column_name = "ner_tags"
label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)

{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [5]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels )

2024-03-13 04:34:54.561561: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 04:34:54.561664: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 04:34:54.736243: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
model=model.to("cuda")

In [7]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [8]:
# # Get the length of the original training set
# original_train_size = len(raw_datasets["train"])

# # Calculate the number of samples to keep for 20% of the original training set
# new_train_size = int(original_train_size * 0.05)
train_dataset = raw_datasets["train"].select(range(20000))
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/5 [00:00<?, ?ba/s]

In [9]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [10]:
data_collator = DataCollatorForTokenClassification(tokenizer)


In [11]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [12]:
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    weight_decay=0.02
)

In [13]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

In [14]:
trainer.args

TrainingArguments(
_n_gpu=2,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_

In [15]:
train_result = trainer.train()
metrics = train_result.metrics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.153100,0.170998,0.810911,0.855772,0.832738,10213,0.681762,0.676885,0.679315,9786,0.805359,0.839042,0.821856,10568,0.769140,0.792718,0.780751,0.947085
2,0.103200,0.182718,0.817899,0.849212,0.833261,10213,0.672278,0.697834,0.684817,9786,0.805091,0.829012,0.816876,10568,0.766749,0.793765,0.780023,0.946727
3,0.074100,0.207823,0.810879,0.850974,0.830443,10213,0.670534,0.685060,0.677719,9786,0.799155,0.823429,0.811111,10568,0.762443,0.788334,0.775172,0.945797


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [16]:
Final_metrics = trainer.evaluate()

trainer.log_metrics("eval", Final_metrics)

***** eval metrics *****
  epoch                   =        3.0
  eval_LOC_f1             =     0.8304
  eval_LOC_number         =      10213
  eval_LOC_precision      =     0.8109
  eval_LOC_recall         =      0.851
  eval_ORG_f1             =     0.6777
  eval_ORG_number         =       9786
  eval_ORG_precision      =     0.6705
  eval_ORG_recall         =     0.6851
  eval_PER_f1             =     0.8111
  eval_PER_number         =      10568
  eval_PER_precision      =     0.7992
  eval_PER_recall         =     0.8234
  eval_loss               =     0.2078
  eval_overall_accuracy   =     0.9458
  eval_overall_f1         =     0.7752
  eval_overall_precision  =     0.7624
  eval_overall_recall     =     0.7883
  eval_runtime            = 0:05:09.78
  eval_samples_per_second =      43.45
  eval_steps_per_second   =      2.718


In [17]:
model.save_pretrained('Finetuned_indicNER_with_20000_sentences_BatchSize=8_weight_1')

In [18]:
tokenizer.save_pretrained("tokenizer_indicNER_BatchSize=8_weight_1")

('tokenizer_indicNER_BatchSize=8_weight_1/tokenizer_config.json',
 'tokenizer_indicNER_BatchSize=8_weight_1/special_tokens_map.json',
 'tokenizer_indicNER_BatchSize=8_weight_1/vocab.txt',
 'tokenizer_indicNER_BatchSize=8_weight_1/added_tokens.json',
 'tokenizer_indicNER_BatchSize=8_weight_1/tokenizer.json')

In [19]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [20]:
import json
config = json.load(open("/kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_weight_1/config.json"))

In [21]:
config["id2label"] = id2label
config["label2id"] = label2id

In [22]:
json.dump(config, open("/kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_weight_1/config.json","w"))

In [23]:
model = AutoModelForTokenClassification.from_pretrained('/kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_weight_1')

In [24]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [35]:
!zip -r IndicNERTuned_20000_BatchSize8weightdecay_best.zip /kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_weight_1

  adding: kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_weight_1/ (stored 0%)
  adding: kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_weight_1/config.json (deflated 50%)
  adding: kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_weight_1/model.safetensors (deflated 7%)


In [36]:
!zip -r IndicNER_tokenizer_20000_BatchSize8_best.zip /kaggle/working/tokenizer_indicNER_BatchSize=8_weight_1

  adding: kaggle/working/tokenizer_indicNER_BatchSize=8_weight_1/ (stored 0%)
  adding: kaggle/working/tokenizer_indicNER_BatchSize=8_weight_1/special_tokens_map.json (deflated 80%)
  adding: kaggle/working/tokenizer_indicNER_BatchSize=8_weight_1/tokenizer_config.json (deflated 75%)
  adding: kaggle/working/tokenizer_indicNER_BatchSize=8_weight_1/vocab.txt (deflated 48%)
  adding: kaggle/working/tokenizer_indicNER_BatchSize=8_weight_1/tokenizer.json (deflated 69%)


In [25]:
test_dataset = raw_datasets["test"]
tokenize_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [26]:
predictions, labels, metrics = trainer.predict(tokenize_test_dataset)

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall'] = metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1'] = metrics[key]

In [27]:
print(metrics)

{'test_loss': 0.17027251422405243, 'test_LOC_precision': 0.797427652733119, 'test_LOC_recall': 0.8078175895765473, 'test_LOC_f1': 0.8025889967637541, 'test_LOC_number': 614, 'test_ORG_precision': 0.6486486486486487, 'test_ORG_recall': 0.7771428571428571, 'test_ORG_f1': 0.7071057192374349, 'test_ORG_number': 525, 'test_PER_precision': 0.8284023668639053, 'test_PER_recall': 0.8860759493670886, 'test_PER_f1': 0.856269113149847, 'test_PER_number': 790, 'test_overall_precision': 0.7652671755725191, 'test_overall_recall': 0.831518921721099, 'test_overall_f1': 0.7970186335403726, 'test_overall_accuracy': 0.9536520384054693, 'test_runtime': 20.8601, 'test_samples_per_second': 41.563, 'test_steps_per_second': 2.637}


In [28]:
print('macro f1 score is ::',metrics['test_overall_f1'])

macro f1 score is :: 0.7970186335403726


In [29]:
print(lang_specific_results)

{'Precision': 0.7652671755725191, 'Recall': 0.831518921721099, 'F1': 0.7970186335403726}


In [33]:
predictions, labels, metrics = trainer.predict(train_dataset)

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall'] = metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1'] = metrics[key]

In [34]:
print(metrics)
print('\n')
print(lang_specific_results)

{'test_loss': 0.048478271812200546, 'test_LOC_precision': 0.912650406504065, 'test_LOC_recall': 0.9454888484603463, 'test_LOC_f1': 0.9287794545935928, 'test_LOC_number': 14841, 'test_ORG_precision': 0.8692416725726435, 'test_ORG_recall': 0.8709700326658145, 'test_ORG_f1': 0.8701049943246311, 'test_ORG_number': 14082, 'test_PER_precision': 0.9176620076238882, 'test_PER_recall': 0.9250672473421289, 'test_PER_f1': 0.9213497480385278, 'test_PER_number': 15614, 'test_overall_precision': 0.9008512990602543, 'test_overall_recall': 0.9147674966881469, 'test_overall_f1': 0.9077560660413093, 'test_overall_accuracy': 0.9837717106085218, 'test_runtime': 459.2142, 'test_samples_per_second': 43.553, 'test_steps_per_second': 2.722}


{'Precision': 0.9008512990602543, 'Recall': 0.9147674966881469, 'F1': 0.9077560660413093}


In [30]:
import torch
def get_ner(sentence):
    tok_sentence = tokenizer(sentence, return_tensors='pt')

    with torch.no_grad():
        logits = model(**tok_sentence).logits.argmax(-1)
        predicted_tokens_classes = [
            model.config.id2label[t.item()] for t in logits[0]]

        predicted_labels = []

        previous_token_id = 0
        word_ids = tok_sentence.word_ids()
        for word_index in range(len(word_ids)):
            if word_ids[word_index] == None:
                previous_token_id = word_ids[word_index]
            elif word_ids[word_index] == previous_token_id:
                previous_token_id = word_ids[word_index]
            else:
                predicted_labels.append(predicted_tokens_classes[word_index])
                previous_token_id = word_ids[word_index]

        ner_output = []
        for index in range(len(sentence.split(' '))):
            if(index<len(predicted_labels)):
                ner_output.append((sentence.split(' ')[index], predicted_labels[index]))
            else:
                ner_output.append((sentence.split(' ')[index], 'O'))
        return ner_output

In [31]:
All_sentences=['नारायण साईं पर जेल में रहते हुए पुलिस कर्मचारी को 13 करोड़ रुपए की रिश्वत देने का भी आरोप लगा था, लेकिन इस मामले में नारायण साईं को जमानत तो मिल चुकी है लेकिन रेप के मामले में अभी भी कोर्ट में सुनवाई चल रही है।',
           'इसके बाद वह अपनी मां से तब तक विडियो कॉल पर बात करने से बचती रहीं तब तक उन्होंने अपने वेट कम नहीं कर लिया।',
           'इससे पहले प्रिंस नरुला ने इस बात की पुष्टि की है कि वह शो की सहभागी एवं मोरक्को की सुंदरी नोरा फातेही के के साथ समय. . . दीप्ति अपहरण मामले पर बोले शाहरुख, लोग फिल्मों से गलत सीख लेते हैं',
           'अखिलेश और राहुल पर हमला करते हुए योगी ने कहा कि शहजादे और युवराज का यूपी पर ध्यान नहीं जाता.',
           'आषाढ़ माह में बारिश होने से तालाबों में मेढकाें की टर्र-टर्र सुनाई पड़ने लगती थी लेकिन इस बार बेल्हा से बादल इस कदर रूठे रहे कि मई और जून माह में सिर्फ एक मिलीमीटर बारिश हो सकी। बरसाती मेढक दिखे ही नहीं।',
           'जानकारी है कि प्रशांत किशोर ने इस काम के लिए राहुल गांधी से कोई भी वेतन नहीं ले रहे हैं.',
           'देवासुर संग्राम में शक्ति (नारी) ही चंडी-दुर्गा के रूप में प्रकट होती है।',
           'इसका दुष्प्रभाव यह है कि जनप्रतिनिधियों से महत्वपूर्ण पार्टियां हो जाती हैं. फिर पार्टी कमान कंपनी का सीइओ हो जाता है और जनप्रतिनिधि उनके द्वारा नियुक्त होते हैं, केवल औपचारिक तौर पर उनका चुनाव जनता के द्वारा होना जरूरी है.',
           'मुंबई: महाराष्ट्र की राजनीति में उठापटक का दौर जारी है.',
           '20/08/2014 मुख्य अभियंता, धसान केन कछार सागर के अन्तर्गत गुण नियंत्रण के कार्यों को गति देने हेतु मुख्यालय परिवर्तन।',
           'हालांकि, पिछले साल दिसंबर से सभी टेलिकॉम कंपनियों ने अपने प्रीपेड प्लान्स में बदलाव किया था, जिसके बाद ये प्रीपेड प्लान डिसकन्टिन्यू कर दिया गया है.',
           'पैनासोनिक ने पेश किए दो स्‍मार्टफोन, कीमत है इनकी 27000 रुपए तक - India TV Paisa',
           'एयर इंडिया को सरकार 2100 करोड़ रु की मदद देगी, 6 साल में 26000 करोड़ मिले फिर भी नहीं सुधरी हालत',
           'पिछले मैच में हमारे साथ जो हुआ उससे टीम ने काफी कुछ सीखा.',
           'सरकारें राजनीतिक दलों की होती हैं और वे जनता के हित में कोई योजना चलाते हैं, जिनमें वायदे ही तो होते हैं।',
           'शपथ ग्रहण से पहले बीजेपी में बड़े बदलाव हो सकते हैं।',
           'युधिष्ठिर की उदारता भी अलौकिक थी । जब कौरवो ने किसी प्रकार भी इनका राज्य लौटाना मंजूर नहीं किया तो इन्होंने केवल पांच गांव लेकर संतोष करना स्वीकार कर लिया और भगवान् श्रीकृष्णके द्वारा दुर्योधन को यह कहला भेजा कि यदि वह हमें हमारे इच्छानुसार केवल पांच गांव देना मंजूर कर ले तो हम युद्ध न करे ।',
           'महंत ने कहा कि अब इंतजार के अलावा कोई विकल्प भी नहीं है।',
           'गौरतलब है कि इस मंदिर के सामने की तरफ कुछ मीटर की दूरी पर ही पुलिस का नाका लगा रहता है।',
           'इस शपथ ग्रहण समारोह में इस बार एक बात यह खास है कि पाकिस्‍तान के पीएम इमरान खान को न्‍योता नहीं दिया गया है।',
           'इसी वजह से मिठाई की जगह पर लोगों ने उपहार देने के लिए ड्राई फ्रूट्स और चौकलेट का प्रयोग करना शुरू किया.',
           'उनका कहना है कि प्रदर्शन के विश्लेषण के लिहाज से कम से कम 3-5 साल का नजरिया रखा जाना चाहिए।',
           'रेंजर्स ने मशहूर पर्यटक स्थल टाफ्ट प्वाइंट से नीचे गुरुवार को दुर्गम इलाके से उनके शव बरामद किये। टाफ्ट प्वाइंट से योसेमिटी घाटी का मनोरम दृश्य देखने को मिलता है।',
           'इसके पहले अपने बयान पर सफाई देते हुए प्रज्ञा सिंह ठाकुर ने ट्वीट किया, कभी-2 झूठ का बबण्डर इतना गहरा होता है कि दिन मे भी रात लगने लगती है किन्तु सूर्य अपना प्रकाश नहीं खोता पलभर के बबण्डर मे लोग भ्रमित न हों सूर्य का प्रकाश स्थाई है।',
           'शायद हमें कभी ना पता चले लेकिन यह बहुत दुखद घटना है। खबर में बताया गया है कि दोनों 2014 से शादीशुदा थे और दोनों सॉफ्टवेयर इंजीनियर थे।'
          ]
IndicBERT_tag=[]
for each_line in All_sentences:
    # print(each_line)
    output=get_ner(each_line)
#     print(output)
    List=[]
    for index in range(len(output)):
        List.append(output[index][1])
        if output[index][0].find(",") != -1 and len(output[index][0])!=1:
            List.append('NULL')
        if output[index][0].find("।") != -1 and len(output[index][0])!=1:
            List.append('NULL')
        if output[index][0].find(".") != -1 and len(output[index][0])!=1:
            List.append('NULL')
        if output[index][0].find(":") != -1 and len(output[index][0])!=1:
            List.append('NULL')
#     print(List)
    IndicBERT_tag.append(List)
print(IndicBERT_tag)

# for line in IndicBERT_tag:
#     print(len(line))


[['B-PER', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'NULL', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'NULL'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'NULL'], ['O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'NULL', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'NULL', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'B-PER', 'O', 'B-PER', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'NULL'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'NULL', 'O', 'O', 'O', 'O', 'O', 